#[색인 & 가중치 => 학습]
#1.수집한 문서목록 가져오기
#2.각문서에서 색인어 추출 전처리 -> 어절 -> 형태소 + 명사 + 바이그램
#3.가중치 계산(TF-IDF) DTM -> TDM -> TWM : 문서단어행렬(DTM:Document Term Matrix), 단어문서행렬(TDM:Term Document Matrix), 단어가중치행렬(Term Weight Matrix)

=======================================================================

#[질의 => 색인과정의 2 TO 3]
#1.질의에서 색인어 추출
#2.가중치 계산
#3.유사도 계산 (Euclidean vs. Cosine)
#4.유사도 순 정렬(거리-오름차순, 각도-내림차순)
#5.검색결과 출력

=======================================================================

#[성능]
#•Precesion(정확율) : 검색결과는 최대한 많이,
#•Recall(재현율) : 유사도는 최대한 높이

#동의 질의로 검색결과 확인 후 퇴근(Q:서울시에 거래되는 아파트의 전세값은?)
#참고 <http://www.reacademy.org/rboard/data/krea2_new/63_4.pdf>

# [색인 & 가중치 => 학습]
## 1.수집한 문서목록 가져오기

In [5]:
from os import listdir

def fileids(path, ext="txt"):
    #fileLIst = list()
    #for file in listdir(path):
    #    if file.split(".")[-1] == ext :
    #        fileList.append(path+file)
    #return fileList
    return [path+file for file in listdir(path) if file.split(".")[-1] == ext]

len(fileids("./news/")),fileids("./news/")

(181,
 ['./news/article001-201905100313.txt',
  './news/article001-201905101012.txt',
  './news/article001-201905101529.txt',
  './news/article001-201905101842.txt',
  './news/article001-201905102001.txt',
  './news/article001-201905102108.txt',
  './news/article001-201905102109.txt',
  './news/article001-201905102119.txt',
  './news/article001-201905102125.txt',
  './news/article002-201905091733.txt',
  './news/article002-201905100609.txt',
  './news/article002-201905102030.txt',
  './news/article002-201905102100.txt',
  './news/article002-201905102105.txt',
  './news/article002-201905102111.txt',
  './news/article002-201905102119.txt',
  './news/article002-201905102126.txt',
  './news/article002-201905102133.txt',
  './news/article003-201905091629.txt',
  './news/article003-201905100336.txt',
  './news/article003-201905100740.txt',
  './news/article003-201905101200.txt',
  './news/article003-201905101542.txt',
  './news/article003-201905101916.txt',
  './news/article003-201905101917.

## 2.각문서에서 색인어 추출 전처리 -> 어절 -> 형태소 + 명사 + 바이그램


In [6]:
def filecontent(file) :
    with open(file, encoding="utf-8") as fp :
        content = fp.read()
    return content

def ngram(term, n=2) :
    return [term[i:i+n] for i in range(len(term) - n + 1)]


In [7]:
from string import punctuation
import re

pattern = dict()

# 구두점
pattern["punc"] = re.compile(r"[{0}]".format(re.escape(punctuation)))

# 불용어
pattern["stopword"] = re.compile(r"[A-Za-z0-9]{7,}")

# Email \w{2,}@\w{3,}(.\w{2,})
pattern["email"] = re.compile(r"\w{2,}@\w{3,}(.\w{2,})")

# 도메인
pattern["domain"] = re.compile(r"(\.?\w{2,}){2,}")

# 한글이외
pattern["nonkorea"] = re.compile(r"[^가-힣0-9]+")

# 반복 공백문자
pattern["whitespace"] = re.compile(r"\s{2,}")

In [9]:
from collections import defaultdict
#원시어절, 구두점, 불용어, 길이, 빈도, 정규식, 품사분석(형태소), ngram
from nltk.tokenize import word_tokenize
from konlpy.tag import Komoran

ma = Komoran()

indexTerm = defaultdict(int)
indexTerm1 = defaultdict(int)
indexTerm2 = defaultdict(int)
indexTerm3 = defaultdict(int)
indexTerm4 = defaultdict(int)
indexTerm5 = defaultdict(int)

for file in fileids("./news/") :
    print(file)
    content = filecontent(file)
    
    for _ in ["punc", "stopword", "email", "domain", "nonkorea", "whitespace"] :
        content = pattern[_].sub(" ", content)

    # 원시어절
    for term in word_tokenize(content) :
        indexTerm1[term] += 1  # 원시 어절
        indexTerm[term] += 1  

    # 토플    
    for _ in indexTerm1 :
        for t in ma.pos(_) :
            indexTerm2[t] += 1  # 원시형태소 + 품사
            indexTerm[t[0]] += 1  
            if len(t[0]) > 1 :   # 음절 길이로 정규화
                indexTerm3[t[0]] += 1  # 원시형태소
                indexTerm[t[0]] += 1  
            if t[1].startswith("N"):
                indexTerm4[t] += 1  # 명사
                indexTerm[t[0]] += 1  
            for n in ngram(t[0]) :   # Bigram
                indexTerm5[n] += 1  # 
                indexTerm[n] += 1  

len(indexTerm1), len(indexTerm2), len(indexTerm3), len(indexTerm4), len(indexTerm), \
min(indexTerm5.values()),max(indexTerm5.values())
#indexTerm  

./news/article001-201905100313.txt
./news/article001-201905101012.txt
./news/article001-201905101529.txt
./news/article001-201905101842.txt
./news/article001-201905102001.txt
./news/article001-201905102108.txt
./news/article001-201905102109.txt
./news/article001-201905102119.txt
./news/article001-201905102125.txt
./news/article002-201905091733.txt
./news/article002-201905100609.txt
./news/article002-201905102030.txt
./news/article002-201905102100.txt
./news/article002-201905102105.txt
./news/article002-201905102111.txt
./news/article002-201905102119.txt
./news/article002-201905102126.txt
./news/article002-201905102133.txt
./news/article003-201905091629.txt
./news/article003-201905100336.txt
./news/article003-201905100740.txt
./news/article003-201905101200.txt
./news/article003-201905101542.txt
./news/article003-201905101916.txt
./news/article003-201905101917.txt
./news/article003-201905102009.txt
./news/article003-201905102030.txt
./news/article003-201905102100.txt
./news/article004-20

(9493, 5225, 4285, 4026, 12639, 4, 5914)

In [10]:
collection = list()
for file in fileids("./news/") :
    content = filecontent(file)
    collection.append((file, content))

globalDocument = list()
globalPosting = list()
waitPosting = list()
documentLength = list(0.0 for _ in range(len(collection)))

globalLexicon = dict()
globalMaxTF = dict()
globalTotalTF = dict()    

#len(collection)
len(collection), collection

(181,
 [('./news/article001-201905100313.txt',
   '택시 동승·렌터카 합승 일단 보류{\'기자\': \'김봉기 기자\', \'입력\': \'201905100313\'}\n모빌리티 스타트업들이 준비한 택시 동승 중개 서비스와 6~10인승 렌터카를 활용한 합승 운행 서비스가 9일 \'규제 벽\'을 넘지 못하고 규제 샌드박스 대상에서 보류됐다. "정부가 생색낼 만한 규제만 빨리 풀고 정작 민감한 사안에 대해선 결정을 미루면서 스스로 규제 샌드박스의 의미를 퇴색시킨다"는 지적이 나온다. 규제 샌드박스는 어린이가 모래 놀이터에서 마음껏 뛰노는 것처럼 규제를 일정 기간 면제하거나 유예해 새로운 서비스가 시장에 신속히 출시되도록 돕는 제도다.\n\n\n\n과학기술정보통신부는 이날 3차 신기술·서비스 심의위원회를 열고 ICT(정보통신기술) 분야의 규제 샌드박스 5건을 논의했다. 하지만 이 중 2건은 재논의하기로 하고 3건만 통과시켰다.\n◇택시 동승 중개, 6~10인승 렌터카 합승 등 처리 보류\n보류된 안건은 스마트폰 앱을 활용해 ▲행선지가 비슷한 승객 2명을 연결해주는 택시 중개 서비스(요금은 승객 2명이 절반씩 부담), ▲공항~대도시 간을 6~10인승 렌터카 및 6~13인승 대형 택시로 오가는 합승 서비스다. 현행법에선 택시 합승이나 10인승 이하 렌터카의 운전자를 알선하는 행위가 금지되어 있다.\n과기정통부 관계자는 "회의 때 민간 위원은 적극적으로 승인하자는 의견이 많았지만, 정부 측 인사들은 \'사회적 합의가 선행됐으면 좋겠다\'는 의견을 내놨다"며 "이에 다음 회의로 결론을 미룬 것"이라고 말했다. 국토부 등에선 합승 허용으로 발생할 수 있는 부작용이 없는지 더 검토해야 한다는 입장을 보였다고 한다. 이에 대해 "택시 업계와 밀접한 관련이 있는 사안들인 만큼 선뜻 결론을 내리지 않은 것 아니냐"는 지적이 적지 않다. 반대 의견 중에는 렌터카에 운전자를 알선해주는 행위는 현재 11인승 이상 차량에만 가능한데, 이를 6~10인

In [11]:
for docName, docContent in collection:
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    #print(docIdx, docName, globalDocument.index(docName))
    
    localPosting = defaultdict(int)
    for token in word_tokenize(docContent) :
        for term in ma.pos(token) :
            if len(term[0]) > 1 and term[1].startswith("N") :
                localPosting[term[0]] += 1
    
    globalMaxTF[docIdx] = max(localPosting.values())
    globalTotalTF[docIdx] = sum(localPosting.values())
    
    ## local 끝
    #print(globalMaxTF)
    #print("#################################")
    #print(globalTotalTF)
    
    for term,freq in localPosting.items() :
        if term in globalLexicon.keys():
            ## Lexicon term ? =>
            #termIdx = list(globalLexicon.keys()).index(term)
            postingIdx = len(globalPosting)
            globalPosting.append((docIdx, freq, globalLexicon[term]))
            globalLexicon[term] = postingIdx
        else :
            termIdx = len(globalLexicon.keys())
            postingIdx = len(globalPosting)
            globalLexicon[term] = postingIdx
            globalPosting.append((docIdx, freq, -1))


In [31]:
## TF
# Term Frequency (특정 단어가 문서내에 얼만큼 나타나는지 빈도)
# Inverse Document Frequency(전체문서의 수 / 단어 t가 포함된 문서의 수)
# 특정 문서 내 특정 단어의 빈도 = f(t,d)
# 특정 문서 내 다른 단어의 빈도 => sum, max => DTM(sum,max)
from math import log10
# log2 < log10

def logTF(freq) :
    return log10(1+freq)

def totalTF(freq, totalFreq) :
    return freq/totalFreq

# alpha = 0[0-1] , query = 0.5[0.5-1]
def doubleTF(freq, maxFreq, alpha=0.5):
    return alpha+(1-alpha)*(freq/maxFreq)

In [32]:
## IDF
# 전체 문서의 갯수 => N
# 특정 단어가 나타난 문서의 갯수 => df(t) => 

# is a the of => stopwords ==> 0 
# to be or not to be = 0
def idf(df,N) :
    return log10(N/df)

#  0 이상 나옴
def smoothingIdf(df, N) :
    return log10(N/1+df)

# 특정기준값을 바탕으로 부호가 바뀜
def probabilisticIdf(df, N) :
    return log10((N-df+1)/df)

In [34]:
## TF-IDF
N = len(globalDocument)

for term, postingIdx in globalLexicon.items() : # TDM.keys()
    old = postingIdx
    df = 0 
    
    while True :
        if postingIdx == -1 :
            break
        df += 1
        postingIdx = globalPosting[postingIdx][2]
    
    postingIdx = old
    idf1 = idf(df, N)
        
    while True :
        if postingIdx == -1 :
            break
        
        data = globalPosting[postingIdx]
        postingIdx = data[2]
        tf = doubleTF(data[1], globalMaxTF[data[0]])
#        data[1] = tf * idf1
#        documentLength[data[0]] += tf*idf1
        #(0.5 + 0.5(data[1]/globalMaxTF[data[0]]))* log10(N/df)
        print('{0}-{1}'.format(term, globalDocument[data[0]]))
        print(' => {0} * {1} = {2}'.format(tf, idf1, tf*idf1))

#len(globalLexicon)
#len(globalDocument)
#len(globalPosting)
#len(globalMaxTF)
#len(globalTotalTF)

택시-./news/article020-201905102042.txt
 => 0.6875 * 1.354588587877241 = 0.9312796541656032
택시-./news/article012-201905101125.txt
 => 0.7 * 1.354588587877241 = 0.9482120115140686
택시-./news/article011-201905102042.txt
 => 0.6875 * 1.354588587877241 = 0.9312796541656032
택시-./news/article009-201905101125.txt
 => 0.7 * 1.354588587877241 = 0.9482120115140686
택시-./news/article004-201905091000.txt
 => 0.8636363636363636 * 1.354588587877241 = 1.1698719622576172
택시-./news/article003-201905091629.txt
 => 0.7692307692307692 * 1.354588587877241 = 1.0419912214440314
택시-./news/article002-201905091733.txt
 => 1.0 * 1.354588587877241 = 1.354588587877241
택시-./news/article001-201905100313.txt
 => 0.95 * 1.354588587877241 = 1.2868591584833788
동승-./news/article012-201905101125.txt
 => 0.65 * 1.479527324485541 = 0.9616927609156016
동승-./news/article009-201905101125.txt
 => 0.65 * 1.479527324485541 = 0.9616927609156016
동승-./news/article004-201905091000.txt
 => 0.7272727272727273 * 1.479527324485541 = 1.0760198

입장-./news/article017-201905102106.txt
 => 0.5714285714285714 * 0.5252848150462159 = 0.3001627514549805
입장-./news/article017-201905102015.txt
 => 0.525 * 0.5252848150462159 = 0.2757745278992634
입장-./news/article016-201905102021.txt
 => 0.5714285714285714 * 0.5252848150462159 = 0.3001627514549805
입장-./news/article016-201905102012.txt
 => 0.5416666666666666 * 0.5252848150462159 = 0.28452927481670026
입장-./news/article016-201905100005.txt
 => 0.5344827586206896 * 0.5252848150462159 = 0.28075567700746024
입장-./news/article015-201905102105.txt
 => 0.5294117647058824 * 0.5252848150462159 = 0.2780919609068202
입장-./news/article015-201905102040.txt
 => 0.65 * 0.5252848150462159 = 0.34143512978004037
입장-./news/article015-201905102028.txt
 => 1.0 * 0.5252848150462159 = 0.5252848150462159
입장-./news/article015-201905100953.txt
 => 0.5869565217391304 * 0.5252848150462159 = 0.3083193479619093
입장-./news/article014-201905102039.txt
 => 0.5833333333333334 * 0.5252848150462159 = 0.30641614211029267
입장-./new

 => 0.5384615384615384 * 0.8105205435269653 = 0.4364341388222121
부분-./news/article005-201905102100.txt
 => 0.5454545454545454 * 0.8105205435269653 = 0.44210211465107196
부분-./news/article005-201905101531.txt
 => 0.5384615384615384 * 0.8105205435269653 = 0.4364341388222121
부분-./news/article004-201905101500.txt
 => 0.53125 * 0.8105205435269653 = 0.43058903874870036
부분-./news/article004-201905092231.txt
 => 0.5714285714285714 * 0.8105205435269653 = 0.46315459630112304
부분-./news/article003-201905100740.txt
 => 0.625 * 0.8105205435269653 = 0.5065753397043533
부분-./news/article002-201905102100.txt
 => 0.5454545454545454 * 0.8105205435269653 = 0.44210211465107196
부분-./news/article002-201905102030.txt
 => 0.5555555555555556 * 0.8105205435269653 = 0.4502891908483141
부분-./news/article002-201905091733.txt
 => 0.5277777777777778 * 0.8105205435269653 = 0.4277747313058984
부분-./news/article001-201905101012.txt
 => 0.5166666666666667 * 0.8105205435269653 = 0.4187689474889321
작업-./news/article020-2019051

비난-./news/article001-201905102119.txt
 => 0.5238095238095238 * 1.5587085705331658 = 0.8164663940888012
비난-./news/article001-201905101012.txt
 => 0.5166666666666667 * 1.5587085705331658 = 0.8053327614421357
안무-./news/article002-201905100609.txt
 => 0.5833333333333334 * 1.9566485792052033 = 1.141378337869702
안무-./news/article001-201905101012.txt
 => 0.5166666666666667 * 1.9566485792052033 = 1.0109350992560218
소리-./news/article021-201905102022.txt
 => 0.55 * 1.2162858897109594 = 0.6689572393410277
소리-./news/article020-201905102022.txt
 => 0.55 * 1.2162858897109594 = 0.6689572393410277
소리-./news/article016-201905100005.txt
 => 0.5172413793103449 * 1.2162858897109594 = 0.6291133912298066
소리-./news/article012-201905101646.txt
 => 0.5172413793103449 * 1.2162858897109594 = 0.6291133912298066
소리-./news/article012-201905101348.txt
 => 0.5263157894736842 * 1.2162858897109594 = 0.640150468268926
소리-./news/article011-201905102022.txt
 => 0.55 * 1.2162858897109594 = 0.6689572393410277
소리-./news/arti

혐의-./news/article016-201905102012.txt
 => 0.5416666666666666 * 0.6666139678426853 = 0.3610825659147879
혐의-./news/article016-201905101144.txt
 => 0.7142857142857143 * 0.6666139678426853 = 0.4761528341733467
혐의-./news/article015-201905102105.txt
 => 0.5588235294117647 * 0.6666139678426853 = 0.3725195702650301
혐의-./news/article015-201905102040.txt
 => 0.55 * 0.6666139678426853 = 0.366637682313477
혐의-./news/article015-201905102018.txt
 => 0.5416666666666666 * 0.6666139678426853 = 0.3610825659147879
혐의-./news/article015-201905101348.txt
 => 0.8181818181818181 * 0.6666139678426853 = 0.5454114282349243
혐의-./news/article014-201905102039.txt
 => 0.8333333333333333 * 0.6666139678426853 = 0.5555116398689044
혐의-./news/article014-201905101908.txt
 => 0.5217391304347826 * 0.6666139678426853 = 0.3477985919179228
혐의-./news/article012-201905102040.txt
 => 0.55 * 0.6666139678426853 = 0.366637682313477
혐의-./news/article010-201905101216.txt
 => 0.7105263157894737 * 0.6666139678426853 = 0.4736467666250659


...

연구원-./news/article017-201905102015.txt
 => 0.525 * 1.479527324485541 = 0.776751845354909
연구원-./news/article014-201905102026.txt
 => 0.5192307692307693 * 1.479527324485541 = 0.7682161107905694
연구원-./news/article013-201905100849.txt
 => 0.65 * 1.479527324485541 = 0.9616927609156016
연구원-./news/article011-201905091759.txt
 => 0.6818181818181819 * 1.479527324485541 = 1.0087686303310508
연구원-./news/article005-201905102106.txt
 => 0.5555555555555556 * 1.479527324485541 = 0.8219596247141895
연구원-./news/article004-201905092231.txt
 => 0.5714285714285714 * 1.479527324485541 = 0.8454441854203091
본부장-./news/article016-201905102021.txt
 => 0.5714285714285714 * 1.5587085705331658 = 0.8906906117332376
본부장-./news/article014-201905101908.txt
 => 0.5217391304347826 * 1.5587085705331658 = 0.8132392541912169
본부장-./news/article010-201905102021.txt
 => 0.5714285714285714 * 1.5587085705331658 = 0.8906906117332376
본부장-./news/article008-201905101908.txt
 => 0.5217391304347826 * 1.5587085705331658 = 0.81323925419

 => 0.55 * 1.479527324485541 = 0.8137400284670475
보험-./news/article008-201905102022.txt
 => 0.55 * 1.479527324485541 = 0.8137400284670475
보험-./news/article005-201905102111.txt
 => 0.5294117647058824 * 1.479527324485541 = 0.7832791717864629
구축-./news/article020-201905101921.txt
 => 0.5384615384615384 * 1.5587085705331658 = 0.8393046149024739
구축-./news/article008-201905102023.txt
 => 0.5277777777777778 * 1.5587085705331658 = 0.8226517455591709
구축-./news/article007-201905101706.txt
 => 0.7142857142857143 * 1.5587085705331658 = 1.113363264666547
구축-./news/article006-201905101716.txt
 => 0.5333333333333333 * 1.5587085705331658 = 0.8313112376176884
구축-./news/article005-201905102111.txt
 => 0.5294117647058824 * 1.5587085705331658 = 0.8251986549881466
증시-./news/article019-201905101940.txt
 => 0.5416666666666666 * 1.5587085705331658 = 0.8443004757054647
증시-./news/article013-201905100849.txt
 => 0.6 * 1.5587085705331658 = 0.9352251423198994
증시-./news/article012-201905101756.txt
 => 0.5625 * 1.55

윤관석-./news/article007-201905100631.txt
 => 0.5416666666666666 * 1.9566485792052033 = 1.0598513137361518
이재정-./news/article013-201905100631.txt
 => 0.5416666666666666 * 1.9566485792052033 = 1.0598513137361518
이재정-./news/article007-201905100631.txt
 => 0.5416666666666666 * 1.9566485792052033 = 1.0598513137361518
제윤경-./news/article013-201905100631.txt
 => 0.5416666666666666 * 1.9566485792052033 = 1.0598513137361518
제윤경-./news/article007-201905100631.txt
 => 0.5416666666666666 * 1.9566485792052033 = 1.0598513137361518
정춘숙-./news/article013-201905100631.txt
 => 0.5833333333333334 * 1.9566485792052033 = 1.141378337869702
정춘숙-./news/article007-201905100631.txt
 => 0.5833333333333334 * 1.9566485792052033 = 1.141378337869702
홍의락-./news/article013-201905100631.txt
 => 0.5416666666666666 * 1.9566485792052033 = 1.0598513137361518
홍의락-./news/article007-201905100631.txt
 => 0.5416666666666666 * 1.9566485792052033 = 1.0598513137361518
이개호-./news/article013-201905100631.txt
 => 0.5416666666666666 * 1.

목숨-./news/article015-201905101147.txt
 => 0.55 * 1.479527324485541 = 0.8137400284670475
목숨-./news/article014-201905102112.txt
 => 0.7 * 1.479527324485541 = 1.0356691271398786
목숨-./news/article012-201905101147.txt
 => 0.55 * 1.479527324485541 = 0.8137400284670475
목숨-./news/article011-201905102112.txt
 => 0.7 * 1.479527324485541 = 1.0356691271398786
목숨-./news/article008-201905102103.txt
 => 0.5416666666666666 * 1.479527324485541 = 0.8014106340963346
애도-./news/article008-201905102103.txt
 => 0.5833333333333334 * 2.2576785748691846 = 1.3169791686736911
플로-./news/article010-201905101357.txt
 => 0.53125 * 1.9566485792052033 = 1.0394695577027642
플로-./news/article008-201905102103.txt
 => 0.5416666666666666 * 1.9566485792052033 = 1.0598513137361518
랑스-./news/article008-201905102103.txt
 => 0.5416666666666666 * 2.2576785748691846 = 1.2229092280541416
테러리스트-./news/article008-201905102103.txt
 => 0.5416666666666666 * 2.2576785748691846 = 1.2229092280541416
배후-./news/article008-201905102103.txt
 =>

반박-./news/article017-201905102106.txt
 => 0.5714285714285714 * 1.4125805348549276 = 0.8071888770599586
반박-./news/article016-201905101924.txt
 => 0.5909090909090909 * 1.4125805348549276 = 0.8347066796870027
반박-./news/article015-201905102040.txt
 => 0.6 * 1.4125805348549276 = 0.8475483209129565
반박-./news/article013-201905101924.txt
 => 0.5909090909090909 * 1.4125805348549276 = 0.8347066796870027
반박-./news/article012-201905102040.txt
 => 0.6 * 1.4125805348549276 = 0.8475483209129565
분량-./news/article015-201905102040.txt
 => 0.55 * 1.9566485792052033 = 1.076156718562862
분량-./news/article012-201905102040.txt
 => 0.55 * 1.9566485792052033 = 1.076156718562862
납득-./news/article016-201905101924.txt
 => 0.5454545454545454 * 1.655618583541222 = 0.9030646819315756
납득-./news/article015-201905102040.txt
 => 0.55 * 1.655618583541222 = 0.9105902209476722
납득-./news/article013-201905101924.txt
 => 0.5454545454545454 * 1.655618583541222 = 0.9030646819315756
납득-./news/article012-201905102040.txt
 => 0.55 

In [ ]:
#globalLexicon
#globalDocument
#globalPosting
#globalMaxTF
#globalTotalTF